# Modeling

### Imports

In [3]:
import pandas as pd

#### Read in Preprocessed Data

In [4]:
# Restaurant Menu Data
menu_df = pd.read_csv('./preprocessed_data/menu_df.csv')
menu_df.head()

,restaurant_name,food_name,serving_size,serving_unit,calories,carbohydrates,sugars,fats,saturated_fats,cholesterol,sodium,fiber,potassium,proteins,carb_percent,fat_percent,protein_percent,score
0,McDonald's Canada,Egg BLT McMuffin with Shredded Lettuce (McDona...,1.0,Serving,7.99,1.55,0.56,0.14,0.02,0.00,3.76,0.99,116.09,0.58,77.596996,15.769712,29.036295,0.308
1,McDonald's,Cheeseburger,1.0,Serving,535.31,39.24,7.16,28.66,14.00,95.52,1176.09,2.39,443.77,30.27,29.321328,48.185164,22.618670,0.519
2,McDonald's,Hamburger,1.0,Serving,540.14,40.27,7.16,26.56,10.52,122.04,791.00,0.03,569.52,34.28,29.821898,44.255193,25.386011,0.498
3,McDonald's,Honey,1.0,Serving,63.84,17.30,17.25,0.00,0.00,0.00,0.84,0.04,10.92,0.06,108.395990,0.000000,0.375940,0.166
4,McDonald's,Hotcakes,1.0,Serving,90.80,11.32,7.16,3.88,0.85,23.60,175.60,0.03,52.80,2.56,49.867841,38.458150,11.277533,0.135


In [5]:
# Individual Food Data
individual_foods_df = pd.read_csv('./preprocessed_data/individual_foods_df.csv')

individual_foods_df.head()

,food_name,category,description,food_category,calories,carbohydrates,fiber,sugars,fats,proteins,score
0,Almond Butter,Individual Foods,ALMOND BUTTER,Nut & Seed Butters,587.8,22.50,11.00,6.992,50.98,21.320,0.4612
1,Almond Flour,Individual Foods,ALMOND FLOUR,Flours & Corn Meal,604.8,19.64,9.22,6.348,52.08,21.180,0.4304
2,Almonds,Individual Foods,ALMONDS,"Popcorn, Peanuts, Seeds & Related Snacks",594.8,18.12,11.90,3.142,51.90,20.660,0.5462
3,Apples,Individual Foods,APPLES,Pre-Packaged Fruit & Vegetables,52.8,14.18,2.76,10.480,0.00,0.554,0.1746
4,Asparagus,Individual Foods,ASPARAGUS,Pre-Packaged Fruit & Vegetables,21.0,3.92,2.36,2.076,0.00,2.076,0.2960


In [6]:
# Patient Data
patient_df = pd.read_csv('./preprocessed_data/patient_df.csv')
patient_df.head()

,ID,Diabetes_012,HighBP,HighChol,CholCheck,BMI,Smoker,Stroke,HeartDiseaseorAttack,PhysActivity,GenHlth,MentHlth,PhysHlth,Sex,Age,Glucose Value,Time Checked,GlucoseRank,Cluster
0,16515,2.0,0,1.0,1,33.0,0.0,1.0,1.0,1,5.0,30.0,30.0,1,0.833333,87,2024-09-08T19:38:12,Norm,3
1,12855,2.0,0,1.0,0,30.0,1.0,0.0,1.0,0,4.0,30.0,15.0,0,0.416667,129,2024-09-21T14:43:19,Norm,0
2,6691,2.0,0,0.0,0,31.0,0.0,0.0,1.0,1,5.0,4.0,0.0,0,0.500000,152,2024-10-13T04:33:24,High,0
3,15303,2.0,0,0.0,1,34.0,1.0,0.0,0.0,0,2.0,0.0,0.0,0,0.583333,99,2024-09-13T00:38:24,Norm,3
4,5714,2.0,0,1.0,1,25.0,1.0,0.0,0.0,1,3.0,0.0,0.0,0,0.500000,115,2024-10-16T13:58:32,Norm,3


### Compute Scores for Recommendations

While we already have a score that captures the suitability of a meal/food item based on general diabetes nutrition recommendations, this patient-specific score uses that general score in combination with personalized factors to suggest the best meal options for a person.

In [7]:
# Patient-Specific Score Function
def patient_suitability_score(patient_data, food_data):
    # Patient Info
    glucose_rank = patient_data['GlucoseRank']
    high_bp = patient_data['HighBP']
    high_chol = patient_data['HighChol']
    smoker = patient_data['Smoker']
    stroke = patient_data['Stroke']
    heart = patient_data['HeartDiseaseorAttack']
    general_health = patient_data['GenHlth']

    # Score Adjustments
    score = food_data['score']
    
    # High glucose lowers the suitability score
    if glucose_rank == 'High':
        score -= 0.1
    # Low glucose raises the suitability score
    elif glucose_rank == 'Low':
        score += 0.1
    else:
        score = score

    # Health Penalties
    score -= 0.05 * (high_bp + high_chol + smoker + stroke + heart)

    patient_score = max(0, score)
    return patient_score

#### Menu Data Patient Scores

In [8]:
# Initialize empty df for recommendations
menu_recs = []

patient_sample = patient_df.sample(n=1000)

# Run each patient/menu food pair through the function to calculate a score
for _, patient in patient_sample.iterrows():
    for _, food in menu_df.iterrows():
        score = patient_suitability_score(patient, food)
        menu_recs.append({
            'Patient_ID': patient['ID'],
            'Food_Name': food['food_name'],
            'Food_Category': None,
            'Restaurant': food['restaurant_name'],
            'General_Score': food['score'],
            'Patient_Score': score
        })

# Convert to pandas DataFrame
menu_recs_df = pd.DataFrame(menu_recs)

# Print
menu_recs_df.head()

,Patient_ID,Food_Name,Food_Category,Restaurant,General_Score,Patient_Score
0,6671,Egg BLT McMuffin with Shredded Lettuce (McDona...,None,McDonald's Canada,0.308,0.058
1,6671,Cheeseburger,None,McDonald's,0.519,0.269
2,6671,Hamburger,None,McDonald's,0.498,0.248
3,6671,Honey,None,McDonald's,0.166,0.000
4,6671,Hotcakes,None,McDonald's,0.135,0.000


#### Individual Food Patient Scores

In [9]:
# Initialize empty df for recommendations
individual_food_recs = []

# Run each patient/individual food pair through the function to calculate a score
for _, patient in patient_sample.iterrows():
    for _, food in individual_foods_df.iterrows():
        score = patient_suitability_score(patient, food)
        individual_food_recs.append({
            'Patient_ID': patient['ID'],
            'Food_Name': food['food_name'],
            'Food_Category': food['food_category'],
            'Restaurant': None,
            'General_Score': food['score'],
            'Patient_Score': score
        })

# Convert to pandas DataFrame
individual_food_recs_df = pd.DataFrame(individual_food_recs)

# Print
individual_food_recs_df.head()

,Patient_ID,Food_Name,Food_Category,Restaurant,General_Score,Patient_Score
0,6671,Almond Butter,Nut & Seed Butters,None,0.4612,0.2112
1,6671,Almond Flour,Flours & Corn Meal,None,0.4304,0.1804
2,6671,Almonds,"Popcorn, Peanuts, Seeds & Related Snacks",None,0.5462,0.2962
3,6671,Apples,Pre-Packaged Fruit & Vegetables,None,0.1746,0.0000
4,6671,Asparagus,Pre-Packaged Fruit & Vegetables,None,0.2960,0.0460


#### Recommend Top 5 Meals and Top 5 Foods per Patient

In [19]:
# Group by Patient_ID and get the top 5 Patient_Score for menu recommendations
top_menu_recs = menu_recs_df.groupby('Patient_ID').apply(
    lambda group: group.nlargest(5, 'Patient_Score')
).reset_index(drop=True)

# Display the result
top_menu_recs.head()

C:\Users\tarad\AppData\Local\Temp\ipykernel_25228\878498078.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  top_menu_recs = menu_recs_df.groupby('Patient_ID').apply(


,Patient_ID,Food_Name,Food_Category,Restaurant,General_Score,Patient_Score
0,52,"House Cheesesteak, Beef",None,Yard House,0.882,0.782
1,52,Chicken Drunken Noodles,None,Pick Up Stix,0.852,0.752
2,52,Black Beans,None,Chipotle,0.843,0.743
3,52,Black Beans,None,Chili's,0.843,0.743
4,52,Black Beans,None,Broken Yolk Cafe,0.843,0.743


In [20]:
# Group by Patient_ID and get the top 5 Patient_Score for menu recommendations
top_foods_recs = individual_food_recs_df.groupby('Patient_ID').apply(
    lambda group: group.nlargest(5, 'Patient_Score')
).reset_index(drop=True)

# Display the result
top_foods_recs.head()

C:\Users\tarad\AppData\Local\Temp\ipykernel_25228\3211321186.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  top_foods_recs = individual_food_recs_df.groupby('Patient_ID').apply(


,Patient_ID,Food_Name,Food_Category,Restaurant,General_Score,Patient_Score
0,52,Chia seeds,Other Grains & Seeds,None,0.8940,0.7940
1,52,Lentils,Vegetable and Lentil Mixes,None,0.7916,0.6916
2,52,Rye Bread,Breads & Buns,None,0.7910,0.6910
3,52,Flax seeds,Other Grains & Seeds,None,0.7700,0.6700
4,52,Farro,Other Grains & Seeds,None,0.7128,0.6128


In [21]:
# Combine all food recommendations
final_recs = pd.concat([top_menu_recs, top_foods_recs], ignore_index=True)

# Display the result
final_recs.head()

,Patient_ID,Food_Name,Food_Category,Restaurant,General_Score,Patient_Score
0,52,"House Cheesesteak, Beef",None,Yard House,0.882,0.782
1,52,Chicken Drunken Noodles,None,Pick Up Stix,0.852,0.752
2,52,Black Beans,None,Chipotle,0.843,0.743
3,52,Black Beans,None,Chili's,0.843,0.743
4,52,Black Beans,None,Broken Yolk Cafe,0.843,0.743
